In [148]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import export_graphviz
import graphviz

train = pd.read_csv('train.csv')


def bar_chart(feature):  #대략적인 생존자와 사망자 파악을 위해 차트 그래프 
    Survived = train[train['Survived'] == 1][feature].value_counts()
    Dead = train[train['Survived'] == 0][feature].value_counts()
    df = pd.DataFrame([Survived,Dead])
    df.index = ['Survived','Dead']
    df.plot(kind ='bar', stacked = True)
    
    
def dummy_process(data, columns):
    for column in columns:
        data = pd.concat([data, pd.get_dummies(data[column], prefix = column)], axis=1)
        data = data.drop(column, axis=1)
    return data
    
    
for data in train: #나이에 대한 결측치가 많기 때문에 여러 feature중 나이를 어느정도 유추할 수 있는 근거로 이름을 사용
    train['Name_part'] = train.Name.str.extract('([A-Za-z]+)\.',expand = False)  


    
train['Name_part'] = train['Name_part'].replace(['Capt','Col'],'Minor')             
train['Name_part'] = train['Name_part'].replace(['Ms','Lady'],'Miss')
train['Name_part'] = train['Name_part'].replace(['Mme','Mlle','Countess'],'Mrs')
train['Name_part'] = train['Name_part'].replace(['Rev','Jonkheer','Major','Sir','Don'],'Mr')    #소수의 이름은 다수 이름으로 편입시킴


for N_partition,age in train.groupby('Name_part')['Age'].median().iteritems():
    train.loc[(train['Name_part']==N_partition) & (train['Age'].isnull()), 'Age'] = age

    

    
train = train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis = 1)  # 고려하지 않은 정보 삭제






train = train.fillna({"Embarked" : 'S'})  #다수가 S이기 때문에 S로 간주하고 데이터 처리





embarked_number = {'S' : 1, 'Q' : 2, 'C' : 3}                      #embarked 넘버링으로 맵핑
train['Embarked'] = train['Embarked'].map(embarked_number)
test['Embarked'] = test['Embarked'].map(embarked_number)


dummy_features = ['Sex','Pclass','Embarked']
train = dummy_process(train, dummy_features)





train['Age'] = train['Age'].fillna(-0.5)


bins = [-1, 0, 5, 13, 19, 60 , 200]                                                #결측치 마이너스로 넣어두고 나이별로 그룹화 맵핑한다.
labels = ['UNK', 'Baby', 'Child', 'Teenager','Adult','Senior']
train['Age_Group'] = pd.cut(train['Age'], bins, labels = labels)

age_number = {'UNK':0, 'Baby' : 1,'Child' : 2, 'Teenager' : 3, 'Adult' : 4, 'Senior':5}
train['Age_Group'] = train['Age_Group'].map(age_number)
train = train.drop(['Age'], axis = 1)

train['Fare'] = pd.qcut(train['Fare'], 5, labels = [1,2,3,4,5])  #Fare 값이 편차가 크긴 하지만 요금에 따른 생존 여부가 지대한 영향을 미친다는 근거가 부족해
                                                                 #5 그룹으로 단순히 나누었다.
    
    
train = train.drop('Name_part', axis =1 )

train['Family'] = train['Parch']+train['SibSp']   # 두 특성은 같이 가족으로 고려해도 되기 때문에 family로 합치고 두 특성 삭제
train = train.drop('SibSp',axis = 1)
train = train.drop('Parch',axis = 1)



x = train.drop('Survived', axis =1)
y = train['Survived']

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state = 40)   #train.csv 파일을 7:3으로 나누어서 테스트


def train_model(model):
    model.fit(x_train,y_train)
    prediction = model.predict(x_test)
    accuracy = accuracy_score(prediction,y_test)
    return accuracy

def train_model_f1 (model):
    model.fit(x_train,y_train)
    prediction = model.predict(x_test)
    f1 = f1_score(prediction,y_test)
    return f1


train_for_log = train
train_for_log = train_for_log.drop(['Fare','Age_Group','Family'],axis=1)



print("================KNN Result================")
print("KNN accuaracy(K = 1) : ",train_model(KNeighborsClassifier(n_neighbors = 1)),"    KNN f1(K = 1) : ",train_model_f1(KNeighborsClassifier(n_neighbors = 1)))
print("KNN accuaracy(K = 2) : ",train_model(KNeighborsClassifier(n_neighbors = 2)),"    KNN f1(K = 2) : ",train_model_f1(KNeighborsClassifier(n_neighbors = 2)))
print("KNN accuaracy(K = 3) : ",train_model(KNeighborsClassifier(n_neighbors = 3)),"    KNN f1(K = 3) : ",train_model_f1(KNeighborsClassifier(n_neighbors = 3)))
print("KNN accuaracy(K = 4) : ",train_model(KNeighborsClassifier(n_neighbors = 4)),"    KNN f1(K = 4) : ",train_model_f1(KNeighborsClassifier(n_neighbors = 4)))
print("KNN accuaracy(K = 5) : ",train_model(KNeighborsClassifier(n_neighbors = 5)),"    KNN f1(K = 5) : ",train_model_f1(KNeighborsClassifier(n_neighbors = 5)))

print("\n")

print("============Logistic Regression============")
print("Logistic Regression accuracy(iter==20) : " ,train_model(LogisticRegression(max_iter=20)),"    Logistic Regression f1 : ",train_model_f1(LogisticRegression(max_iter=20)))
print("Logistic Regression accuracy(iter==40) : ", train_model(LogisticRegression(max_iter=40)),"    Logistic Regression f1 : ",train_model_f1(LogisticRegression(max_iter=40)))
print("Logistic Regression accuracy(iter==60) : ", train_model(LogisticRegression(max_iter=60)),"    Logistic Regression f1 : ",train_model_f1(LogisticRegression(max_iter=60)))
print("Logistic Regression accuracy(iter==80) : ", train_model(LogisticRegression(max_iter=80)),"    Logistic Regression f1 : ",train_model_f1(LogisticRegression(max_iter=80)))
print("Logistic Regression accuracy(iter==100) : ", train_model(LogisticRegression(max_iter=100)),"    Logistic Regression f1 : ",train_model_f1(LogisticRegression(max_iter=100)))
print("Logistic Regression accuracy(iter==100,C=1) : ", train_model(LogisticRegression(max_iter=100,C=1)),"    Logistic Regression f1 : ",train_model_f1(LogisticRegression(max_iter=100,C=1)))
print("Logistic Regression accuracy(iter==100,C=2) : ", train_model(LogisticRegression(max_iter=100,C=2)),"    Logistic Regression f1 : ",train_model_f1(LogisticRegression(max_iter=100,C=2)))
print("Logistic Regression accuracy(iter==100,C=3) : ", train_model(LogisticRegression(max_iter=100,C=3)),"    Logistic Regression f1 : ",train_model_f1(LogisticRegression(max_iter=100,C=3)))
print("Logistic Regression accuracy(iter==100,C=4) : ", train_model(LogisticRegression(max_iter=100,C=4)),"    Logistic Regression f1 : ",train_model_f1(LogisticRegression(max_iter=100,C=4)))
print("Logistic Regression accuracy(iter==100,C=5) : ", train_model(LogisticRegression(max_iter=100,C=5)),"    Logistic Regression f1 : ",train_model_f1(LogisticRegression(max_iter=100,C=5)))

print("\n")

print("==============Decision Tree==============")
print("Decision Tree accuracy : ",train_model(DecisionTreeClassifier(random_state=30)),"    Decision Tree f1 : ",train_model_f1(DecisionTreeClassifier(random_state=30)))

print("\n")

bag_model=BaggingClassifier(base_estimator=DecisionTreeClassifier(),random_state=30,n_estimators=1)

print("========Decision Tree with Bagging========")
print("Decision Tree with Bagging accuracy(n=1) : ",train_model(BaggingClassifier(base_estimator=DecisionTreeClassifier(),random_state=30,n_estimators=1)),"Decision Tree with Bagging f1(n=1) : ",train_model_f1(BaggingClassifier(base_estimator=DecisionTreeClassifier(),random_state=30,n_estimators=1)))
print("Decision Tree with Bagging accuracy(n=2) : ",train_model(BaggingClassifier(base_estimator=DecisionTreeClassifier(),random_state=30,n_estimators=2)),"Decision Tree with Bagging f1(n=2) : ",train_model_f1(BaggingClassifier(base_estimator=DecisionTreeClassifier(),random_state=30,n_estimators=2)))
print("Decision Tree with Bagging accuracy(n=3) : ",train_model(BaggingClassifier(base_estimator=DecisionTreeClassifier(),random_state=30,n_estimators=3)),"Decision Tree with Bagging f1(n=3) : ",train_model_f1(BaggingClassifier(base_estimator=DecisionTreeClassifier(),random_state=30,n_estimators=3)))
print("Decision Tree with Bagging accuracy(n=4) : ",train_model(BaggingClassifier(base_estimator=DecisionTreeClassifier(),random_state=30,n_estimators=4)),"Decision Tree with Bagging f1(n=4) : ",train_model_f1(BaggingClassifier(base_estimator=DecisionTreeClassifier(),random_state=30,n_estimators=4)))
print("Decision Tree with Bagging accuracy(n=5) : ",train_model(BaggingClassifier(base_estimator=DecisionTreeClassifier(),random_state=30,n_estimators=5)),"Decision Tree with Bagging f1(n=5) : ",train_model_f1(BaggingClassifier(base_estimator=DecisionTreeClassifier(),random_state=30,n_estimators=5)))

================KNN Result================
KNN accuaracy(K = 1) :  0.6977611940298507     KNN f1(K = 1) :  0.6582278481012659
KNN accuaracy(K = 2) :  0.7611940298507462     KNN f1(K = 2) :  0.6666666666666666
KNN accuaracy(K = 3) :  0.8022388059701493     KNN f1(K = 3) :  0.7511737089201879
KNN accuaracy(K = 4) :  0.8059701492537313     KNN f1(K = 4) :  0.7425742574257426
KNN accuaracy(K = 5) :  0.8432835820895522     KNN f1(K = 5) :  0.8108108108108109


============Logistic Regression============
Logistic Regression accuracy(iter==20) :  0.8134328358208955     Logistic Regression f1 :  0.7727272727272728
Logistic Regression accuracy(iter==40) :  0.8134328358208955     Logistic Regression f1 :  0.7727272727272728
Logistic Regression accuracy(iter==60) : 

C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regre

 0.8134328358208955     Logistic Regression f1 :  0.7727272727272728
Logistic Regression accuracy(iter==80) :  0.8134328358208955     Logistic Regression f1 :  0.7727272727272728
Logistic Regression accuracy(iter==100) :  0.8134328358208955     Logistic Regression f1 :  0.7727272727272728
Logistic Regression accuracy(iter==100,C=1) :  0.8134328358208955     Logistic Regression f1 :  0.7727272727272728
Logistic Regression accuracy(iter==100,C=2) :  0.8134328358208955     Logistic Regression f1 :  0.7727272727272728
Logistic Regression accuracy(iter==100,C=3) :  0.8134328358208955     Logistic Regression f1 :  0.7727272727272728
Logistic Regression accuracy(iter==100,C=4) :  0.8134328358208955     Logistic Regression f1 :  0.7727272727272728
Logistic Regression accuracy(iter==100,C=5) :  0.8134328358208955     Logistic Regression f1 :  0.7727272727272728


==============Decision Tree==============
Decision Tree accuracy :  0.8134328358208955     Decision Tree f1 :  0.766355140186916


==